In [2]:
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()

# model = ChatGroq(model="deepseek-r1-distill-llama-70b", reasoning_format="parsed")
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
model

ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x73d6f4cb1a60>, default_metadata=(), model_kwargs={})

In [ ]:
message = [
    SystemMessage(content = "You are a helpful assistant."),
    HumanMessage(content = "What is the place city to visit in Europe")
]

message.append(model.invoke(message))

In [21]:
def query(content):
    message.append(HumanMessage(content = content))
    result = model.invoke(message)
    message.append(result)
    return result.content

query("What was my first question ?")

'Your first question was: **"What is the place city to visit in Europe"**. You were asking for recommendations on the best cities to visit in Europe, and I provided a detailed list of top destinations! Let me know if you\'d like more details about any of them. 😊'

In [23]:
query("I am Apoorv student of AIML")

'Hello, Apoorv! 👋 It’s great to meet you. As a student of AI and Machine Learning (AIML), you must be diving into some fascinating topics like neural networks, algorithms, and maybe even cutting-edge technologies like generative AI or reinforcement learning. If you have any questions about your studies, projects, or need help with anything, feel free to ask—I’m here to assist! 🚀\n\nHow’s your journey in AIML going so far? 😊'

In [24]:
query("Who am I again ?")

'You are **Apoorv**, a student of **AI and Machine Learning (AIML)**! 😊 How can I assist you today? Whether it’s about your studies, projects, or anything else, feel free to ask!'

In [27]:
if len(message) > 10:
    message = message[:1]

In [28]:
message

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={})]

## Message History

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [11]:
config = {
    "configurable": {
        "session_id":"chat1"
        }
    }

In [12]:
text = {"content": "Hello, how are you?"}
response = with_message_history.invoke(text, config)
print(response)


content="I'm doing great, thank you for asking!\n\nHow can I help you today?" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--380fa355-aafa-4444-b162-82cdfb76e7fe-0' usage_metadata={'input_tokens': 7, 'output_tokens': 19, 'total_tokens': 117, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 91}}


In [13]:
with_message_history.invoke({"content": "Vale Vale"}, config).content

'Vale, vale!\n\nHow can I help you then?'

In [14]:
with_message_history.invoke(
    [HumanMessage(content = "How to say i love you in french")], 
    config = config
).content

'To say "I love you" in French, you say:\n\n**Je t\'aime**\n\n*   **Je** (I)\n*   **t\'** (you - contraction of "te" before a vowel)\n*   **aime** (love)'

### MessagePlaceholder is used to define the variable for the message in the system

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. answer every question in {line_count} line."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [16]:
response = chain.invoke({"messages": [HumanMessage(content="What is the capital of France?")], "line_count": 1})
print(response.content)

Paris is the capital of France.


In [17]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key= "messages")

In [18]:
config = {"configurable": 
            {
              "session_id": "chat2"
            }
        }

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What is the capital of France?")], 
        "line_count": 1
    },
    config=config
)

response.content

'Paris is the capital of France.'

## Managing the conversation history

-Trim messages: helps to redue howmany messages we are sending to the model. Helps us to specify how many tokens to keep

In [19]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, trim_messages
trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [20]:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the place city to visit in Europe"),
    AIMessage(content="Paris is a great city to visit in Europe."),
    HumanMessage(content="What is the capital of France?"),
    AIMessage(content="The capital of France is Paris."),
    HumanMessage(content="What is the capital of Germany?"),
    AIMessage(content="The capital of Germany is Berlin."),
    HumanMessage(content="What is the capital of Italy?"),
    AIMessage(content="The capital of Italy is Rome."),
    HumanMessage(content="What is the capital of Spain?"),
    AIMessage(content="The capital of Spain is Madrid."),
]

In [21]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer) | prompt | model
)

result = chain.invoke({
    "messages": messages + [HumanMessage(content="What is the capital of India?")],
    "line_count": 1
})

result.content

'The capital of India is New Delhi.'

#### Vector Stores and retrievers

In [22]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Dogs are great companions and are known for their loyalty and affection.",
             metadata = {"source": "mammals-docs"}
             ),
    Document(page_content="Cats are independent animals and are known for their agility and curiosity.",
             metadata = {"source": "mammals-docs"}
             ),
    Document(page_content="Birds are known for their ability to fly and their beautiful songs.",
             metadata = {"source": "birds-docs"}
             ),
    Document(page_content="Fish are aquatic animals and are known for their colorful scales and fins.",
             metadata = {"source": "fish-docs"}
             ),
]

In [23]:
print(documents)

[Document(metadata={'source': 'mammals-docs'}, page_content='Dogs are great companions and are known for their loyalty and affection.'), Document(metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.'), Document(metadata={'source': 'birds-docs'}, page_content='Birds are known for their ability to fly and their beautiful songs.'), Document(metadata={'source': 'fish-docs'}, page_content='Fish are aquatic animals and are known for their colorful scales and fins.')]


In [25]:
## Vector Store
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
model

ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x73d6ec1e75f0>, default_metadata=(), model_kwargs={})

In [31]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-l6-v2")

In [34]:
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,)
vector_store

In [37]:
results = vector_store.get(include=["embeddings"])
all_vectors = results.get("embeddings")
first_vector = all_vectors[0]
print(first_vector)

[-2.38108393e-02  3.91990831e-03  9.50494632e-02  7.88813755e-02
 -8.37091878e-02  7.07838610e-02  5.31210331e-03 -3.57250907e-02
  3.84816341e-02  2.29484681e-02  4.09078710e-02  9.67599731e-03
  4.01636958e-02  4.90465313e-02  1.51777044e-02  9.77447815e-03
  2.44302093e-03 -4.37710173e-02 -1.54315280e-02 -2.94769276e-02
 -1.71753541e-01 -4.84478772e-02  2.24018749e-02 -2.58498196e-03
 -1.02492221e-01 -9.99812875e-03  5.00942394e-02 -9.33647305e-02
  7.61535438e-03  3.68562108e-03 -5.92397414e-02  1.98839139e-02
  2.09417362e-02  5.09361736e-02 -6.06974475e-02  3.17423679e-02
  2.71954387e-02  1.93714034e-02  5.28518483e-03  2.82021835e-02
  1.46919917e-02  2.53355317e-02  6.87923059e-02 -4.79911864e-02
 -7.66359568e-02 -3.43609266e-02 -7.73351565e-02 -4.53878678e-02
 -1.82662830e-02 -3.46984491e-02 -4.10790183e-03  5.71112223e-02
  9.20985732e-03  9.98270139e-03  1.21411225e-02 -3.08774430e-02
 -2.81252712e-02 -7.59686381e-02 -2.10256707e-02  7.88249355e-03
  5.29428646e-02  7.85778

In [40]:
vector_store.similarity_search("cat")

[Document(id='3272762d-28ec-4d13-99a3-c238b8aaa4d6', metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.'),
 Document(id='a51cca6e-5a85-4ec8-b0d0-2cb2e9a86886', metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.'),
 Document(id='84687f97-cdca-495e-b7c7-cc51723ad5b7', metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.'),
 Document(id='db900b93-7e8b-4176-a240-ab191cc91581', metadata={'source': 'mammals-docs'}, page_content='Dogs are great companions and are known for their loyalty and affection.')]

In [ ]:
# Async Query
await vector_store.asimilarity_search("cat")

[Document(id='3272762d-28ec-4d13-99a3-c238b8aaa4d6', metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.'),
 Document(id='a51cca6e-5a85-4ec8-b0d0-2cb2e9a86886', metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.'),
 Document(id='84687f97-cdca-495e-b7c7-cc51723ad5b7', metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.'),
 Document(id='db900b93-7e8b-4176-a240-ab191cc91581', metadata={'source': 'mammals-docs'}, page_content='Dogs are great companions and are known for their loyalty and affection.')]

### Retrievers

In [45]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vector_store.similarity_search).bind(k=2)
retriever.batch(["cat", "dog"])

[[Document(id='3272762d-28ec-4d13-99a3-c238b8aaa4d6', metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.'),
  Document(id='84687f97-cdca-495e-b7c7-cc51723ad5b7', metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.')],
 [Document(id='cf55771c-200c-4f2f-8b9b-4c062620adcb', metadata={'source': 'mammals-docs'}, page_content='Dogs are great companions and are known for their loyalty and affection.'),
  Document(id='8d7cb903-31c5-4e0c-b62e-5a96372c4c49', metadata={'source': 'mammals-docs'}, page_content='Dogs are great companions and are known for their loyalty and affection.')]]

In [46]:
vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)
retriever.batch(["cat", "dog"])

[[Document(id='3272762d-28ec-4d13-99a3-c238b8aaa4d6', metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.'),
  Document(id='84687f97-cdca-495e-b7c7-cc51723ad5b7', metadata={'source': 'mammals-docs'}, page_content='Cats are independent animals and are known for their agility and curiosity.')],
 [Document(id='cf55771c-200c-4f2f-8b9b-4c062620adcb', metadata={'source': 'mammals-docs'}, page_content='Dogs are great companions and are known for their loyalty and affection.'),
  Document(id='8d7cb903-31c5-4e0c-b62e-5a96372c4c49', metadata={'source': 'mammals-docs'}, page_content='Dogs are great companions and are known for their loyalty and affection.')]]

In [47]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

message = """
Answer the question based on the context below. {question}
Context: {context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model
response = rag_chain.invoke("tell me about dogs")
response.content

'Based on the provided context, dogs are great companions and are known for their loyalty and affection.'